In [1]:
import os

import japanize_matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bokeh.io import output_notebook

from JapanHorseRaceAnalytics.utilities.base import get_spark_session, read_hive_table
from JapanHorseRaceAnalytics.utilities.structured_logger import logger

# Set pandas display options
pd.set_option("display.max_rows", 1050)
pd.set_option("display.max_columns", 200)

# Set seed for reproducibility
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
random_state = 42

output_notebook()

Loading BokehJS ...

In [2]:
spark = get_spark_session()

24/03/02 18:08:02 WARN Utils: Your hostname, Hanks-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.40.105 instead (on interface en0)
24/03/02 18:08:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/02 18:08:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from JapanHorseRaceAnalytics.utilities.base import get_data_dir

table_name = "features_20240217_v1"
schema = "jhra_curated"
save_path = get_data_dir() / "sql_tables" / f"{table_name}.snappy.parquet"
spark_df = spark.read.table(f"{schema}.{table_name}")

# spark_df.write.mode("overwrite").parquet(str(save_path))
# return pd.read_parquet(save_path)

24/03/02 18:08:44 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/03/02 18:08:44 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/03/02 18:08:45 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


In [9]:
from pyspark.sql import functions as f
spark.conf.set("spark.sql.session.timeZone", "Asia/Tokyo")
spark_df.where(f.col("meta_レースキー") == "06131503").show()

+---------------+---------+---------+-----------+-------------+---------------+-------------+---------------+-------------------------+-------------------+-------------+----------+--------+-----------------------+---------------+-----------------------+---------------+-------------------+-------------------+-------------------+--------------------------------+--------------------------------+-------------------+-----------------------+--------+----------------+--------------+--------------+--------------+------------------+----------------+----------------+----------------+------------------+----------+--------+--------+--------------+--------------+-------------------------------+-------------------------+---------------------------------+-----------------------------+-----------------------------+--------------+------------+------------------+------------------+------------+------------------+--------------+--------------------+----------------+------------------+------------------+-

In [10]:
spark.version

'3.5.0'

In [6]:
spark_df.where(f.col("meta_レースキー") == "06131503").write.mode("overwrite").parquet(
    "test.parquet"
)

In [14]:
df1 = pd.read_parquet("test.parquet")
# set meta_発走日時 timezone to Asia/Tokyo
df1["meta_発走日時"] = (
    df1["meta_発走日時"].dt.tz_localize("UTC").dt.tz_convert("Asia/Tokyo")
)
df1

,meta_レースキー,meta_馬番,meta_着順,meta_本賞金,meta_単勝的中,meta_単勝払戻金,meta_複勝的中,meta_複勝払戻金,meta_int_races_レースキー,meta_発走日時,meta_場コード,cat_四半期,cat_距離,cat_事前_馬場状態コード,num_事前_馬場差,cat_実績_馬場状態コード,num_実績_馬場差,cat_レース条件_記号,cat_レース条件_条件,cat_レース条件_重量,cat_レース条件_トラック情報_内外,cat_レース条件_トラック情報_右左,cat_レース条件_種別,cat_レース条件_グレード,num_頭数,cat_トラック種別,cat_馬場状態内,cat_馬場状態中,cat_馬場状態外,num_直線馬場差最内,num_直線馬場差内,num_直線馬場差中,num_直線馬場差外,num_直線馬場差大外,cat_芝種類,num_草丈,cat_転圧,cat_凍結防止剤,num_中間降水量,meta_int_race_horses_レースキー,meta_int_race_horses_馬番,meta_int_race_horses_血統登録番号,meta_int_race_horses_発走日時,meta_int_race_horses_異常区分,num_事前ＩＤＭ,cat_事前脚質,num_事前単勝オッズ,num_事前複勝オッズ,cat_事前馬体,cat_事前気配コード,cat_事前上昇度,cat_事前クラスコード,num_事前テン指数,num_事前ペース指数,num_事前上がり指数,num_負担重量,num_馬体重,num_馬体重増減,cat_性別,cat_トラック種別瞬発戦好走馬,cat_トラック種別消耗戦好走馬,num_一走前不利,num_二走前不利,num_三走前不利,num_一走前着順,num_二走前着順,num_三走前着順,num_四走前着順,num_五走前着順,num_六走前着順,num_1走前上昇度,num_2走前上昇度,num_3走前上昇度,num_4走前上昇度,num_5走前上昇度,num_騎手指数,num_情報指数,num_オッズ指数,num_パドック指数,num_総合指数,cat_馬具変更情報,cat_脚元情報,cat_見習い区分,cat_オッズ印,cat_パドック印,cat_直前総合印,cat_距離適性,num_ローテーション,num_基準オッズ,num_基準人気順位,num_基準複勝オッズ,num_基準複勝人気順位,num_特定情報◎,num_特定情報○,num_特定情報▲,num_特定情報△,num_特定情報×,num_総合情報◎,num_総合情報○,num_総合情報▲,...,num_競争相手平均調教師1位完走,num_競争相手調教師1位完走標準偏差,num_競争相手最高調教師トップ3完走,num_競争相手最低調教師トップ3完走,num_競争相手平均調教師トップ3完走,num_競争相手調教師トップ3完走標準偏差,num_競争相手最高調教師1位完走率,num_競争相手最低調教師1位完走率,num_競争相手平均調教師1位完走率,num_競争相手調教師1位完走率標準偏差,num_競争相手最高調教師トップ3完走率,num_競争相手最低調教師トップ3完走率,num_競争相手平均調教師トップ3完走率,num_競争相手調教師トップ3完走率標準偏差,num_競争相手最高調教師場所レース数,num_競争相手最低調教師場所レース数,num_競争相手平均調教師場所レース数,num_競争相手調教師場所レース数標準偏差,num_競争相手最高調教師場所1位完走,num_競争相手最低調教師場所1位完走,num_競争相手平均調教師場所1位完走,num_競争相手調教師場所1位完走標準偏差,num_競争相手最高調教師場所トップ3完走,num_競争相手最低調教師場所トップ3完走,num_競争相手平均調教師場所トップ3完走,num_競争相手調教師場所トップ3完走標準偏差,num_競争相手最高調教師場所1位完走率,num_競争相手最低調教師場所1位完走率,num_競争相手平均調教師場所1位完走率,num_競争相手調教師場所1位完走率標準偏差,num_競争相手最高調教師場所トップ3完走率,num_競争相手最低調教師場所トップ3完走率,num_競争相手平均調教師場所トップ3完走率,num_競争相手調教師場所トップ3完走率標準偏差,num_競争相手最高調教師本賞金累計,num_競争相手最低調教師本賞金累計,num_競争相手平均調教師本賞金累計,num_競争相手調教師本賞金累計標準偏差,num_競争相手最高調教師1位完走平均賞金,num_競争相手最低調教師1位完走平均賞金,num_競争相手平均調教師1位完走平均賞金,num_競争相手調教師1位完走平均賞金標準偏差,num_競争相手最高調教師レース数平均賞金,num_競争相手最低調教師レース数平均賞金,num_競争相手平均調教師レース数平均賞金,num_競争相手調教師レース数平均賞金標準偏差,num_競争相手平均調教師レース数差,num_競争相手平均調教師1位完走差,num_競争相手平均調教師トップ3完走差,num_競争相手平均調教師1位完走率差,num_競争相手平均調教師トップ3完走率差,num_競争相手平均調教師場所レース数差,num_競争相手平均調教師場所1位完走差,num_競争相手平均調教師場所トップ3完走差,num_競争相手平均調教師場所1位完走率差,num_競争相手平均調教師場所トップ3完走率差,num_競争相手平均調教師本賞金累計差,num_競争相手平均調教師1位完走平均賞金差,num_競争相手平均調教師レース数平均賞金差,meta_int_combinations_レースキー,meta_int_combinations_馬番,num_馬騎手レース数,num_馬騎手1位完走,num_馬騎手1位完走率,num_馬騎手トップ3完走,num_馬騎手トップ3完走率,num_馬騎手初二走,num_馬騎手同騎手,num_馬騎手場所レース数,num_馬騎手場所1位完走,num_馬騎手場所1位完走率,num_馬騎手場所トップ3完走,num_馬騎手場所トップ3完走率,num_馬調教師レース数,num_馬調教師1位完走,num_馬調教師1位完走率,num_馬調教師トップ3完走,num_馬調教師トップ3完走率,num_馬調教師初二走,num_馬調教師同調教師,num_馬調教師場所レース数,num_馬調教師場所1位完走,num_馬調教師場所1位完走率,num_馬調教師場所トップ3完走,num_馬調教師場所トップ3完走率,meta_int_race_weather_レースキー,num_temperature,num_precipitation,num_snowfall,num_snow_depth,num_wind_speed,cat_wind_direction,num_solar_radiation,num_local_air_pressure,num_sea_level_air_pressure,num_relative_humidity,num_vapor_pressure,num_dew_point_temperature,cat_weather,num_visibility
0,06131503,13,5,50.0,False,0,False,0,06131503,2013-01-14 02:00:00+00:00,06,1,1200,30,-8,30,-20.0,102,A3,3,1,1,12,None,16,ダート,2,2,2,0,0,0,0,0,None,NaN,False,False,NaN,06131503,13,10106575,2013-01-14 02:00:00,0,20.0,差し,20.9,3.2,None,None,3,38,-25.9,-18.6,-11.6,540,456,0,牝,False,False,0,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,3,None,None,None,None,0.8,2.3,0.0,3.4,26.5,0,2,0,None,2,4,None,3,10.8,5,2.4,5,0,0,3,5,0,8,1,14,...,117.800000,84.218921,753,8,365.800000,237.782590,0.097493,0.015385,0.052432,0.022479,0.281337,0.061538,0.164498,0.056725,942,31,550.533333,300.402811,85,0,34.266667,27.426184,235,1,102.866667,73.035030,0.103542,0.000000,0.055442,0.028723,0.279292,0.032258,0.168717,0.066626,411226.5,3550.0,191385.100000,129101.075009,918.390411,480.000000,777.056566,111.942554,148.688420,27.307692,84.475938,33.078474,904.933333,80.200000,228.200000,0.014573,0.036517,378.466667,21.733333,86.133333,0

In [ ]:
# Something strange is happening when you load dates from parquet?? The race time should be 11:00 for this race.
# Everything is OK in the database.
# df[df["meta_レースキー"] == "06131503"]["meta_発走日時"]


data = read_hive_table(
    table_name="features_20240217_v1",
    schema="jhra_curated",
    spark_session=None,
    # use_cache=False,
)

# drop from data where cat_トラック種別 == "障害"
data = data[(data["cat_トラック種別"] != "障害")]

# drop from data where meta_int_race_horses_異常区分 != '0'
data = data[(data["meta_int_race_horses_異常区分"] == "0")]

# keep only horses that have 3 races
data = data[
    (data["num_一走前着順"].notnull())
    & (data["num_二走前着順"].notnull())
    & (data["num_三走前着順"].notnull())
]

data = data.reset_index(drop=True)
data.head()